# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [3]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType

from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from sklearn import metrics

In [4]:
# create a Spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [5]:
df = spark.read.json('mini_sparkify_event_data.json')
df.persist()
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [6]:
print('Total Records:', "{:,}".format(df.count()))

Total Records: 286,500


In [7]:
# check to see if userId has missing values
df.select("userId").dropDuplicates().sort("userId").show()

+------+
|userId|
+------+
|      |
|    10|
|   100|
|100001|
|100002|
|100003|
|100004|
|100005|
|100006|
|100007|
|100008|
|100009|
|100010|
|100011|
|100012|
|100013|
|100014|
|100015|
|100016|
|100017|
+------+
only showing top 20 rows



Turns out that userId has no missing values, but there are userId values that are empty strings, let's remove them.

In [8]:
# remove empty userIds
df = df.filter(df["userId"] != "")

In [9]:
print('Total Records:', "{:,}".format(df.count()))

Total Records: 278,154


Total Records went from 286,500 to 278,154, that means there were 8,346 userIds with empty strings as values. These recrods are now removed. Let's move on to sessionIds.

In [10]:
# check whether sessionId has missing values
df.select("sessionId").dropDuplicates().sort("sessionId").show()

+---------+
|sessionId|
+---------+
|        1|
|        2|
|        3|
|        4|
|        5|
|        6|
|        7|
|        8|
|        9|
|       10|
|       11|
|       12|
|       13|
|       15|
|       16|
|       17|
|       18|
|       19|
|       20|
|       21|
+---------+
only showing top 20 rows



sessionId has no missing values so no need to remove any more records.

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

I will create a column called 'Churn' which will be for users who have visited the page 'Cancellation Confirmation'.

In [11]:
# how many records are there where users saw the page 'Cancellation Confirmation'
df.select('page').where(df['page'] == 'Cancellation Confirmation').count()

52

In [12]:
# create column called 'Churn' for records where users visited this page
flag_cancel_event = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())
df = df.withColumn("Churn", flag_cancel_event("page"))

df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30', Churn=0)

We have successfully created a 'Churn' column where its value is 1 for events when users cancelled their accounts, 0 otherwise. Now let's do a quick spot check. Earlier we saw that there were 52 records with cancellation events, now let's check how many records we have with 'Churn' == 1.

In [13]:
# number of records with 'Churn' == 1
df.select('Churn').where(df['Churn'] == 1).count()

52

Looks good!

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

Now we need to decide which of the features we would like to keep: which features do we think play a role in users cancelling their accounts.

In [14]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [15]:
# look at all possible values for 'page'
df.select('page').dropDuplicates().show()

+--------------------+
|                page|
+--------------------+
|              Cancel|
|    Submit Downgrade|
|         Thumbs Down|
|                Home|
|           Downgrade|
|         Roll Advert|
|              Logout|
|       Save Settings|
|Cancellation Conf...|
|               About|
|            Settings|
|     Add to Playlist|
|          Add Friend|
|            NextSong|
|           Thumbs Up|
|                Help|
|             Upgrade|
|               Error|
|      Submit Upgrade|
+--------------------+



#### I will go ahead and use the following as features
- number of thumbs up events
- number of thumbs down events
- number of add friend events
- number of add to playlist events
- number of next song events
- number of error events

In order to do that, I'll first need to create a column for each feature so that it'll be easier for me to add these features later on. So for example to get number of thumbs up events for a user I'll traverse throught the dataframe and add a 1 if the user has a thumbs up event and a 0 for all other events. Like this, I'll create the following columns:
- isThumbsUp
- isThumbsDown
- isNextSong
- isAddFriend
- isAddToPlaylist
- isError

In [16]:
# add isThumbsUp column
flag_thumbs_up_event = udf(lambda x: 1 if x == 'Thumbs Up' else 0, IntegerType())
df = df.withColumn('isThumbsUp', flag_thumbs_up_event('page'))
df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30', Churn=0, isThumbsUp=0)

In [17]:
# add isThumbsDown column
flag_thumbs_down_event = udf(lambda x: 1 if x == 'Thumbs Down' else 0, IntegerType())
df = df.withColumn('isThumbsDown', flag_thumbs_down_event('page'))
df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30', Churn=0, isThumbsUp=0, isThumbsDown=0)

In [18]:
# add isNextSong column
flag_next_song_event = udf(lambda x: 1 if x == 'NextSong' else 0, IntegerType())
df = df.withColumn('isNextSong', flag_next_song_event('page'))
df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30', Churn=0, isThumbsUp=0, isThumbsDown=0, isNextSong=1)

In [19]:
# add isAddFriend column
flag_add_friend_event = udf(lambda x: 1 if x == 'Add Friend' else 0, IntegerType())
df = df.withColumn('isAddFriend', flag_add_friend_event('page'))
df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30', Churn=0, isThumbsUp=0, isThumbsDown=0, isNextSong=1, isAddFriend=0)

In [20]:
# add isAddToPlaylist column
flag_add_playlist_event = udf(lambda x: 1 if x == 'Add to Playlist' else 0, IntegerType())
df = df.withColumn('isAddToPlaylist', flag_add_playlist_event('page'))
df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30', Churn=0, isThumbsUp=0, isThumbsDown=0, isNextSong=1, isAddFriend=0, isAddToPlaylist=0)

In [21]:
# add isError column
flag_error_event = udf(lambda x: 1 if x == 'Error' else 0, IntegerType())
df = df.withColumn('isError', flag_error_event('page'))
df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30', Churn=0, isThumbsUp=0, isThumbsDown=0, isNextSong=1, isAddFriend=0, isAddToPlaylist=0, isError=0)

In [22]:
new_df = df.groupBy('userId').sum().select(['sum(isThumbsUp)', 'sum(isThumbsDown)', 'sum(isNextSong)', \
                                            'sum(isAddFriend)', 'sum(isAddToPlaylist)', 'sum(isError)', \
                                            'sum(Churn)'])
new_df.show()

+---------------+-----------------+---------------+----------------+--------------------+------------+----------+
|sum(isThumbsUp)|sum(isThumbsDown)|sum(isNextSong)|sum(isAddFriend)|sum(isAddToPlaylist)|sum(isError)|sum(Churn)|
+---------------+-----------------+---------------+----------------+--------------------+------------+----------+
|             17|                5|            275|               4|                   7|           0|         0|
|             21|                6|            387|               4|                   8|           0|         0|
|              0|                0|              8|               0|                   0|           0|         1|
|            100|               21|           2111|              28|                  52|           1|         1|
|            171|               41|           4079|              74|                 118|           6|         0|
|              7|                1|            150|               1|                   5

Instead of using default column names, let's go ahead and rename our columns to something that makes more sense.

In [23]:
# rename columns
new_df = new_df.select(col("sum(isThumbsUp)").alias("thumbsUp"), \
                       col("sum(isThumbsDown)").alias("thumbsDown"), \
                       col("sum(isNextSong)").alias("nextSong"), \
                       col("sum(isAddFriend)").alias("addFriend"), \
                       col("sum(isAddToPlaylist)").alias("addToPlaylist"), \
                       col("sum(isError)").alias("error"), \
                       col("sum(Churn)").alias("label"))
new_df.printSchema()
new_df.show()

root
 |-- thumbsUp: long (nullable = true)
 |-- thumbsDown: long (nullable = true)
 |-- nextSong: long (nullable = true)
 |-- addFriend: long (nullable = true)
 |-- addToPlaylist: long (nullable = true)
 |-- error: long (nullable = true)
 |-- label: long (nullable = true)

+--------+----------+--------+---------+-------------+-----+-----+
|thumbsUp|thumbsDown|nextSong|addFriend|addToPlaylist|error|label|
+--------+----------+--------+---------+-------------+-----+-----+
|      17|         5|     275|        4|            7|    0|    0|
|      21|         6|     387|        4|            8|    0|    0|
|       0|         0|       8|        0|            0|    0|    1|
|     100|        21|    2111|       28|           52|    1|    1|
|     171|        41|    4079|       74|          118|    6|    0|
|       7|         1|     150|        1|            5|    1|    0|
|     163|        29|    2841|       33|           72|    1|    1|
|      81|        14|    1914|       31|           59|  

Run an analysis on the number of times each User has played a song. What we are looking for here is whether this is a good feature to use, that will determine mostly on its spread of values.

In [24]:
# Analysis on song play events
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

nextSongSeries = new_df.select('nextSong').toPandas()['nextSong']

nextSongSeries.plot.hist(grid=True, bins=20, rwidth=0.9, color='#607c8e')
plt.xlabel('Num Songs Played')
plt.ylabel('Num Users')
plt.grid(axis='y', alpha=0.75)

print('Average', nextSongSeries.mean())
print('Min', nextSongSeries.min())
print('Max', nextSongSeries.max())

Average 1013.81333333
Min 3
Max 8002


That seems like a good spread, there are a few users who've listened to songs more than 3,000 times with the average around a 1,000 and there are other users who've heard way less than 1,000 songs. So there is a decent spread in this data which makes it a good candidate for a feature.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [25]:
train, test, validation = new_df.randomSplit([0.6, 0.2, 0.2], seed=42)

In [26]:
assembler = VectorAssembler(inputCols=['thumbsUp', 'thumbsDown', 'nextSong', 'addFriend', 'addToPlaylist', \
                                       'error'], outputCol="features")

In [33]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=1, maxDepth=2)

In [34]:
pipeline = Pipeline(stages=[assembler, rf])

paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [2, 10]) \
    .addGrid(rf.maxDepth, [2, 10]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=3)

crossval_model = crossval.fit(train)

In [35]:
crossval_model.avgMetrics

[0.6024020815687482,
 0.5986082236082236,
 0.5222514180847514,
 0.5411084161084161]

In [36]:
# Make predictions.
predictions = crossval_model.transform(test)
predictions.show()

+--------+----------+--------+---------+-------------+-----+-----+--------------------+--------------------+--------------------+----------+
|thumbsUp|thumbsDown|nextSong|addFriend|addToPlaylist|error|label|            features|       rawPrediction|         probability|prediction|
+--------+----------+--------+---------+-------------+-----+-----+--------------------+--------------------+--------------------+----------+
|      17|         5|     275|        4|            7|    0|    0|[17.0,5.0,275.0,4...|[1.60804597701149...|[0.80402298850574...|       0.0|
|      11|         0|      84|        3|            1|    0|    0|[11.0,0.0,84.0,3....|[1.60804597701149...|[0.80402298850574...|       0.0|
|      86|        16|    1797|       29|           61|    3|    1|[86.0,16.0,1797.0...|[1.80952380952380...|[0.90476190476190...|       0.0|
|     303|        28|    3632|       63|          113|    5|    0|[303.0,28.0,3632....|[1.80952380952380...|[0.90476190476190...|       0.0|
|      11|   

In [37]:
y_true = list(predictions.select('label').toPandas()['label'])
y_pred = list(predictions.select('prediction').toPandas()['prediction'])

accuracy = metrics.accuracy_score(y_true, y_pred) * 100
print('Accuracy', str(int(accuracy)) + '%')

Accuracy 77%


# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.